In [1]:
import pandas as pd 
import numpy as np 
import math

pulse = pd.read_csv('mri_pulse.csv').to_numpy()
peak = pd.read_csv('resp_pulse.csv').to_numpy()
df = pd.read_csv('rawdata.csv').to_numpy()
df = df[0:7000000]



# Make the equalized histogram of the amplitude

In [2]:
## mark the samples with abnormal value

resp = np.zeros([len(df),2])
resp[:,0] = df[:,1]
resp[:,1] = np.where(resp[:,0] > 0.07, False, True)


In [3]:
max_value = np.max(resp[np.where(resp[:,1] == True),0])
max_value
min_value = np.min(resp[np.where(resp[:,1] == True),0])
min_value
range = max_value - min_value

In [51]:
f = lambda x: (x-min_value)/(range)
resp_norm = f(resp[np.where(resp[:,1] == True),0])
hist = np.histogram(resp_norm, bins =100)
hist


(array([   256,      0,    384,    640,   2816,  10368,  16512,  22528,
         27008,  51456,  59520,  81024,  66304, 115584, 134400, 148736,
        122624, 123008, 151552, 151552, 136576, 116480, 137728, 116224,
        114432,  70016, 117248, 124160, 110208, 100480, 103808, 112256,
        103424,  91776,  95232, 101120,  86656,  63488,  95616,  94464,
         94592,  80256,  91008,  93696,  91904,  79488,  96384,  96000,
        110464,  98048, 105728, 149760, 181248, 154240, 209280, 219008,
        203392, 196352, 203200, 230272, 182656, 161280, 104320, 117632,
         82048,  72832,  46208,  30208,  19968,   6144,   6528,   2688,
           256,    128,      0,    128,      0,    128,    128,      0,
           128,      0,    128,    128,    128,      0,    128,      0,
             0,    128,    128,      0,    128,    256,      0,    256,
             0,    256,      0,    384]),
 array([0.  , 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 ,
        0.11, 0.12, 

In [ ]:
import matplotlib.pyplot as plt

_ = plt.hist(resp_norm, bins=100)
plt.show()

## Find the MRI pulse in the histogram and get the value 

In [22]:
hist = np.asarray(hist)
type(hist)

numpy.ndarray

In [23]:
## Define the function that find the left amplitude

def find_left(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    if array[idx] > value:
        return idx-1
    else:
        return idx

In [52]:
num = len(hist[0])
np.arange(num)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [53]:
## Define the function that calculate the value from histogram A/(A + B)

def phase_value(hist, amp_norm):
    idx = find_left(hist[1],amp_norm)
    num = len(hist[0])
    A = 0
    sum = 0
    for i in np.arange(num):
        if i <= idx:
            A = A + hist[0][i]
        sum = sum + hist[0][i]
    return A/sum

## Find the phase and give + or - to the value

In [93]:
peak[:,0]

array([1.101, 2.918, 3.955, 6.208, 6.246, 7.962, 8.998, 10.906, 10.906,
       10.918, 13.158, 14.925, 14.925, 14.938, 17.498, 17.51, 19.507,
       20.736, 21.453, 23.424, 23.424, 23.437, 25.779, 28.173, 29.491,
       30.31, 30.323, 32.602, 34.125, 34.995, 37.082, 38.656, 39.565,
       41.984, 43.571, 44.454, 46.387, 48.0, 48.922, 51.251, 52.416,
       53.12, 55.834, 55.846, 55.859, 57.19, 57.958, 60.493, 61.747,
       62.566, 64.384, 65.574, 66.33, 68.186, 69.581, 70.477, 72.563,
       73.818, 74.573, 76.851, 78.246, 79.066, 81.05, 81.075, 82.33, 83.2,
       85.338, 86.566, 87.283, 89.114, 90.394, 91.174, 91.2, 93.107,
       94.131, 94.784, 96.691, 98.048, 98.88, 101.005, 102.336, 103.053,
       106.765, 108.544, 109.555, 112.87, 114.598, 115.712, 115.738,
       118.733, 118.758, 118.784, 118.797, 118.797, 118.822, 118.874,
       120.717, 121.715, 121.728, 121.728, 121.741, 121.869, 121.869,
       121.882, 122.048, 122.061, 123.597, 125.222, 126.016, 126.029,
       126.08

In [100]:
## Define the function that find the phase of respiration
## array here is resp_pulse and value here is mri pulse


def find_leftphase(array, value):
    array = np.asarray(array[:,0])
    idx = (np.abs(array - value)).argmin()
    if array[idx] > value:
        return idx-1
    else:
        return idx
        

def find_phase(array, value):
    resp = np.asarray(array)
    sim_resp = resp[np.where(np.equal(resp[:,1],"Recovery") | np.equal(resp[:,1],"Expire Start"))]

    state = peak[find_leftphase(resp,value/10000),1]

    if state == "Inspire Start" :
        state = peak[find_leftphase(sim_resp,value/10000),1]
    if state == "Expire Start":
        return -1
    else:
        return 1
    
    

In [83]:
peak[:,0]

array([1.101, 2.918, 3.955, 6.208, 6.246, 7.962, 8.998, 10.906, 10.906,
       10.918, 13.158, 14.925, 14.925, 14.938, 17.498, 17.51, 19.507,
       20.736, 21.453, 23.424, 23.424, 23.437, 25.779, 28.173, 29.491,
       30.31, 30.323, 32.602, 34.125, 34.995, 37.082, 38.656, 39.565,
       41.984, 43.571, 44.454, 46.387, 48.0, 48.922, 51.251, 52.416,
       53.12, 55.834, 55.846, 55.859, 57.19, 57.958, 60.493, 61.747,
       62.566, 64.384, 65.574, 66.33, 68.186, 69.581, 70.477, 72.563,
       73.818, 74.573, 76.851, 78.246, 79.066, 81.05, 81.075, 82.33, 83.2,
       85.338, 86.566, 87.283, 89.114, 90.394, 91.174, 91.2, 93.107,
       94.131, 94.784, 96.691, 98.048, 98.88, 101.005, 102.336, 103.053,
       106.765, 108.544, 109.555, 112.87, 114.598, 115.712, 115.738,
       118.733, 118.758, 118.784, 118.797, 118.797, 118.822, 118.874,
       120.717, 121.715, 121.728, 121.728, 121.741, 121.869, 121.869,
       121.882, 122.048, 122.061, 123.597, 125.222, 126.016, 126.029,
       126.08

In [108]:
val = []

for i in pulse:
    amp = resp[i,0]
    amp_norm = f(amp)
    val.append(phase_value(hist,amp_norm) * find_phase(peak,i[0]) * math.pi)

val

[-0.33112648044970755,
 -2.9677541247178105,
 -0.5774887195486136,
 2.4837359397841654,
 2.0495131549389747,
 0.20400885206464106,
 -2.9209182896663504,
 -1.1375073852866213,
 -1.3182994553134835,
 -2.9677541247178105,
 0.5094474573388854,
 -2.2001349017731786,
 2.66312006139537,
 0.12234784395651291,
 -2.130886826635744,
 1.8411942567039536,
 1.0457896027809404,
 -2.9209182896663504,
 1.0457896027809404,
 1.4026039584061112,
 2.571890750788999,
 -0.33112648044970755,
 -2.130886826635744,
 1.7548209314495435,
 0.38618013686602476,
 -3.0574030850617087,
 0.940567008941403,
 -2.130886826635744,
 -1.516446644586408,
 -1.4026039584061112,
 3.0205665571255302,
 0.33112648044970755,
 -2.848509514003971,
 0.996310272744981,
 1.0909014193642481,
 3.090101968674323,
 -0.5094474573388854,
 -2.66312006139537,
 -1.3182994553134835,
 -1.2755437973033164,
 3.0205665571255302,
 -0.5774887195486136,
 -3.0574030850617087,
 0.8051166246269357,
 -2.848509514003971,
 0.996310272744981,
 -2.200134901773178

In [110]:
type(val)

list

In [111]:
pd.DataFrame(val).to_csv('resp_phase.csv', index = None)